# generate predictions from snapshots

# setup

In [1]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pathlib import Path

In [2]:
FIGS_DIR = Path(".") / "figs"
FIGS_DIR.mkdir(exist_ok=True)

In [3]:
DATA_DIR = Path(".") / "data"

In [4]:
DATA_DIR.mkdir(exist_ok=True)

In [5]:
SNAPSHOTS_DIR = Path("../../data/") / "snapshots-gen-img"

In [6]:
RECORDS_FPATH = DATA_DIR / "009-generate-predictions-from-snapshots.pkl"

# functions

In [7]:
import numpy as np
from numpy import ndarray
from typing import Dict, Union
from pathlib import Path
import json
import re
from datetime import timedelta, datetime
import numpy as np
from numpy import ndarray
from typing import Dict, Union, List
from pathlib import Path
import json
import numpy as np
from numpy import ndarray
from typing import Dict, Union
from pathlib import Path
import json
import re
import copy
from typing import List


# this is to get the strings associated to the classes in the fcdd code
# copied from: fcdd/python/fcdd/datasets/__init__.py
# inside function `str_labels`
# commit: 9f268d8fd2fee33a5c5f38cdfb781da927bdb614
CLASS_LABELS = {
    'cifar10': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
    'fmnist': [
        't-shirt/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'
    ],
    'mvtec': [
        'bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather',
        'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor',
        'wood', 'zipper'
    ],
    # 'imagenet': deepcopy(ADImageNet.ad_classes),
    # this one forwards to: fcdd/python/fcdd/datasets/imagenet.py
    # in: ADImageNet.ad_classes
    # at the same commit as above
    'imagenet': ['acorn', 'airliner', 'ambulance', 'American alligator', 'banjo', 'barn', 'bikini', 'digital clock',
                  'dragonfly', 'dumbbell', 'forklift', 'goblet', 'grand piano', 'hotdog', 'hourglass', 'manhole cover',
                  'mosque', 'nail', 'parking meter', 'pillow', 'revolver', 'dial telephone', 'schooner',
                  'snowmobile', 'soccer ball', 'stingray', 'strawberry', 'tank', 'toaster', 'volcano'],
    'pascalvoc': ['horse'],
}

def get_classes_labels_order(dataset: str) -> List[str]:
    return copy.deepcopy(CLASS_LABELS[dataset])

def get_class_label(class_dirname: str, dataset: str) -> str:
    return CLASS_LABELS[dataset][int(class_dirname.lstrip("normal_"))]

class MissingFileInExperiment(FileNotFoundError):
    pass

class UnfinishedExperiment(Exception):
    pass

def get_snapshots(experiment_dir: Path) -> Dict[str, Union[str, float, ndarray]]:
    """
    :param experiment_dir: path to the experiment directory
    """
    snapshots_dir = experiment_dir / "training_snapshots"
    
    snapshots = []
    for snashotpath in snapshots_dir.glob("snapshotepoch=*.pt"):
        snapshot_name = snashotpath.name
        epoch = int(snashotpath.stem.split("=")[1]) 
        snapshots.append({
            "fpath": snashotpath,
            "snapshot_name": snapshot_name,
            "epoch": epoch,
        })
        
    return {
        "snapshots": snapshots,
    }


def get_all(path: Path, dataset: str) -> List[Dict[str, Union[str, float, ndarray]]]:
    
    """
    :param path: a folder that contains dirs like 'fcdd_20211220193242_fmnist_' 
                 a whole experiment on a dataset with all iterations and nominal classes inside, 
                 the structure should look like
                 
                path/
                path/normal_0
                path/normal_1
                ...
                path/normal_9/
                path/normal_9/it_0
                ...
                path/normal_9/it_4/roc.json
    """
    assert path.is_dir()
    assert dataset in CLASS_LABELS
    
    experiments = []
    
    # 0th level: the runs, e.g. fcdd_20211220193242_fmnist_
    for rundir in path.glob("*"):
        
        print(f"{rundir.name=}")
        if not rundir.is_dir():
            continue

        # first level: classes
        for classdir in rundir.glob("normal_*"):
            
            print(f"{classdir.name=}")
            if not classdir.is_dir():
                continue

            # second level: iterations
            for iterdir in classdir.glob("it_*"):

                print(f"{iterdir.name=}")
                if not iterdir.is_dir():
                    continue

                try: 
                    experiments.append({
                        **get_snapshots(iterdir),
                        **{
                            "rootdir": path,
                            "rundir": rundir,
                            "classdir": classdir,
                            "iterdir": iterdir,
                            "rundir_name": rundir.name,
                            "classdir_name": classdir.name,
                            "class_idx": int(classdir.name.split("_")[1]),
                            "iterdir_name": iterdir.name,
                            "iter_idx": int(iterdir.name.split("_")[1]),
                        },
                    })

                except UnfinishedExperiment as ex:
                    print(f"experiment in {iterdir=} seems to be unfinished, it'll be ignored")
                    continue
                    
            print(30 * "-")

    return experiments


# get_all(path=SNAPSHOTS_DIR, dataset="mvtec")

# data

In [8]:
import copy
import pickle

try:
    del records
except NameError:
    pass

try:
    print("loading records")
    # pandas cannot reload multindex properly
    # src: https://stackoverflow.com/a/55474405/9582881
    with RECORDS_FPATH.open("r") as f:
        records = pickle.load(f)
except:
    
    print("couldn't find records, recomputing")
    records = get_all(path=SNAPSHOTS_DIR, dataset="mvtec")

    print("saving records")
    with RECORDS_FPATH.open("wb") as f:
        pickle.dump(records, f)
        
f"{len(records)=}"

loading records
couldn't find records, recomputing
rundir.name='fcdd_20220202002148_snapshots2_node24_job3040465__mvtec_'
classdir.name='normal_3'
iterdir.name='it_0'
------------------------------
saving records


'len(records)=1'

# generate one image

I based stuff on the script `fcdd/python/fcdd/runners/run_prediction_with_snapshot.py`.


In [9]:
record = records[0]
snapshots = record["snapshots"]
snapshot = snapshots[0]
epoch = snapshot["epoch"]
fpath = snapshot["fpath"]

In [10]:
import torch
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader

In [11]:
%load_ext autoreload

In [12]:
%autoreload 2
from fcdd.training.fcdd import FCDDTrainer
from fcdd.models.fcdd_cnn_224 import FCDD_CNN224_VGG_F
from fcdd.datasets.image_folder import ImageFolder
from fcdd.datasets.preprocessing import local_contrast_normalization
from fcdd.util.logging import Logger

In [13]:
min_max_l1 = [
    [(-1.3336724042892456, -1.3107913732528687, -1.2445921897888184),
     (1.3779616355895996, 1.3779616355895996, 1.3779616355895996)],
    [(-2.2404820919036865, -2.3387579917907715, -2.2896201610565186),
     (4.573435306549072, 4.573435306549072, 4.573435306549072)],
    [(-3.184587001800537, -3.164201259613037, -3.1392977237701416),
     (1.6995097398757935, 1.6011602878570557, 1.5209171772003174)],
    [(-3.0334954261779785, -2.958242416381836, -2.7701096534729004),
     (6.503103256225586, 5.875098705291748, 5.814228057861328)],
    [(-3.100773334503174, -3.100773334503174, -3.100773334503174),
     (4.27892541885376, 4.27892541885376, 4.27892541885376)],
    [(-3.6565306186676025, -3.507692813873291, -2.7635035514831543),
     (18.966819763183594, 21.64590072631836, 26.408710479736328)],
    [(-1.5192601680755615, -2.2068002223968506, -2.3948357105255127),
     (11.564697265625, 10.976534843444824, 10.378695487976074)],
    [(-1.3207964897155762, -1.2889339923858643, -1.148416519165039),
     (6.854909896850586, 6.854909896850586, 6.854909896850586)],
    [(-0.9883341193199158, -0.9822461605072021, -0.9288841485977173),
     (2.290637969970703, 2.4007883071899414, 2.3044068813323975)],
    [(-7.236185073852539, -7.236185073852539, -7.236185073852539),
     (3.3777384757995605, 3.3777384757995605, 3.3777384757995605)],
    [(-3.2036616802215576, -3.221003532409668, -3.305514335632324),
     (7.022546768188477, 6.115569114685059, 6.310940742492676)],
    [(-0.8915618658065796, -0.8669204115867615, -0.8002046346664429),
     (4.4255571365356445, 4.642300128936768, 4.305730819702148)],
    [(-1.9086798429489136, -2.0004451274871826, -1.929288387298584),
     (5.463134765625, 5.463134765625, 5.463134765625)],
    [(-2.9547364711761475, -3.17536997795105, -3.143850803375244),
     (5.305514812469482, 4.535006523132324, 3.3618252277374268)],
    [(-1.2906527519226074, -1.2906527519226074, -1.2906527519226074),
     (2.515115737915039, 2.515115737915039, 2.515115737915039)]
]

In [14]:
net = FCDD_CNN224_VGG_F((3, 224, 224), bias=True)

In [15]:
from fcdd.datasets import load_dataset
from fcdd.runners.run_mvtec import MvtecConfig
from argparse import ArgumentParser

In [16]:
parser = ArgumentParser()
parser = MvtecConfig()(parser)
dftargs = parser.parse_args("")

In [17]:
dftargs

Namespace(logdir='../../data/results/fcdd_{t}', logdir_suffix='', datadir='../../data/datasets', viz_ids=None, readme='', objective='fcdd', batch_size=16, epochs=200, workers=4, learning_rate=0.001, weight_decay=0.0001, optimizer_type='sgd', scheduler_type='lambda', lr_sched_param=[0.985], load=None, dataset='mvtec', net='FCDD_CNN224_VGG_F', preproc='lcnaug1', acc_batches=8, bias=True, cuda=True, supervise_mode='malformed_normal', noise_mode='confetti', oe_limit=inf, online_supervision=True, nominal_label=0, blur_heatmaps=False, gauss_std=12, quantile=0.99, resdown=64, it=5, cls_restrictions=None, snapshots_training=None)

In [18]:
import copy
r = copy.deepcopy(record)
del r['snapshots']
r

{'rootdir': PosixPath('../../data/snapshots-gen-img'),
 'rundir': PosixPath('../../data/snapshots-gen-img/fcdd_20220202002148_snapshots2_node24_job3040465__mvtec_'),
 'classdir': PosixPath('../../data/snapshots-gen-img/fcdd_20220202002148_snapshots2_node24_job3040465__mvtec_/normal_3'),
 'iterdir': PosixPath('../../data/snapshots-gen-img/fcdd_20220202002148_snapshots2_node24_job3040465__mvtec_/normal_3/it_0'),
 'rundir_name': 'fcdd_20220202002148_snapshots2_node24_job3040465__mvtec_',
 'classdir_name': 'normal_3',
 'class_idx': 3,
 'iterdir_name': 'it_0',
 'iter_idx': 0}

In [19]:
normal_class = record["class_idx"]
normal_class_label = get_class_label(record["classdir_name"], dataset="mvtec")
f"{normal_class_label=}"

"normal_class_label='carpet'"

In [20]:
ds = load_dataset(
    dataset_name="mvtec",
    data_path=DATA_DIR,
    normal_class=normal_class,
    preproc=dftargs.preproc,
    supervise_mode=dftargs.supervise_mode,
    noise_mode=dftargs.noise_mode,
    online_supervision=True,
    nominal_label=dftargs.nominal_label,
    oe_limit=dftargs.oe_limit,
    logger=None,
)

/home/jcasagrandebertoldo/miniconda3/envs/fcdd_rc21/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


Files already downloaded.
Loading dataset from data/mvtec/admvtec_240x240.pt...
Dataset complete.
Files already downloaded.
Loading dataset from data/mvtec/admvtec_240x240.pt...
Dataset complete.


In [21]:
# Use the same test transform as was used for training the snapshot (e.g., for mvtec, per default, the following)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: local_contrast_normalization(x, scale='l1')),
    transforms.Normalize(
        min_max_l1[normal_class][0],
        [ma - mi for ma, mi in zip(min_max_l1[normal_class][1], min_max_l1[normal_class][0])]
    )
])

In [22]:
TEST_IMGS_DIR = Path("../../data/") / "snapshots_test_images" / "carpet"

In [23]:
ds = ImageFolder(TEST_IMGS_DIR, transform, transforms.Lambda(lambda x: 0))

In [24]:
loader = DataLoader(ds, batch_size=16, num_workers=0)

In [48]:
import time

In [49]:
OUTPUTS_DIR = TEST_IMGS_DIR.parent / f"{TEST_IMGS_DIR.name}_preds_{int(time.time())}"
OUTPUTS_DIR

PosixPath('../../data/snapshots_test_images/carpet_preds_1643900192')

In [51]:
# [optional] to generate heatmaps, define a logger (with the path where the heatmaps should be saved to) and a quantile
exp_start_time = int(time.time())
logger = Logger(str(OUTPUTS_DIR), exp_start_time=exp_start_time)
quantile = 0.97

In [52]:
trainer = FCDDTrainer(net, None, None, (None, None), logger, 'fcdd', 8, quantile, 224)

In [53]:
trainer.load(str(snapshot["fpath"]), cpu=True)

Loaded net_state, opt_state, sched_state with starting epoch 49 for fcdd.training.fcdd.FCDDTrainer


49

In [54]:
trainer.net.eval();

In [55]:
all_anomaly_scores, all_inputs, all_labels = [], [], []
for inputs, labels in loader:
    # inputs = inputs.cuda()
    with torch.no_grad():
        outputs = trainer.net(inputs)
        anomaly_scores = trainer.anomaly_score(trainer.loss(outputs, inputs, labels, reduce='none'))
        anomaly_scores = trainer.net.receptive_upsample(anomaly_scores, reception=True, std=8, cpu=False)
        all_anomaly_scores.append(anomaly_scores.cpu())
        all_inputs.append(inputs.cpu())
        all_labels.append(labels)


all_inputs = torch.cat(all_inputs)
all_labels = torch.cat(all_labels)

In [56]:
# all_anomaly_scores will be a tensor containing pixel-wise anomaly scores for all images
all_anomaly_scores = torch.cat(all_anomaly_scores)

# transform the pixel-wise anomaly scores to sample-wise anomaly scores
print(trainer.reduce_ascore(all_anomaly_scores))

tensor([0.0014, 0.0011, 0.0012, 0.0010, 0.0013, 0.0012, 0.0012, 0.0012, 0.0017,
        0.0010, 0.0012, 0.0010, 0.0012, 0.0008, 0.0010, 0.0009, 0.0019, 0.0017,
        0.0021, 0.0013, 0.0013, 0.0012, 0.0011, 0.0011, 0.0013, 0.0016, 0.0013,
        0.0012, 0.0024, 0.0019, 0.0018, 0.0016, 0.0014, 0.0011, 0.0013, 0.0014,
        0.0011, 0.0009, 0.0017, 0.0016, 0.0017, 0.0015, 0.0019, 0.0022, 0.0019,
        0.0019, 0.0008, 0.0009, 0.0020, 0.0017, 0.0016, 0.0019, 0.0011, 0.0009,
        0.0009, 0.0010, 0.0010, 0.0010, 0.0010, 0.0008, 0.0010, 0.0011, 0.0010,
        0.0011, 0.0011, 0.0010, 0.0010, 0.0014, 0.0011, 0.0015, 0.0017, 0.0014,
        0.0013, 0.0011, 0.0014, 0.0013, 0.0013, 0.0014, 0.0012, 0.0013, 0.0011,
        0.0016, 0.0016, 0.0015, 0.0014, 0.0028, 0.0027, 0.0029, 0.0027, 0.0027,
        0.0024, 0.0025, 0.0020, 0.0011, 0.0015, 0.0010, 0.0011, 0.0013, 0.0012,
        0.0009])


In [57]:
# show_per_cls defines the maximum number of samples in the heatmaps figures.
# The heatmap_paper_xxx.png figures, which sort the heatmaps by their anomaly score,
# use only up to a third of show_per_cls samples.
trainer.heatmap_generation(all_labels.tolist(), all_anomaly_scores, all_inputs, show_per_cls=1000)

# putting it into a function

In [ ]:
record = records[0]
snapshots = record["snapshots"]
snapshot = snapshots[0]
epoch = snapshot["epoch"]
fpath = snapshot["fpath"]

In [ ]:
from pathlib import Path

def generate_preds(snapshot_fpath: Path) -> 

never mind this notebook, let's move forward to a cleaner one (010)